In [21]:
import sys
import os
import wget
import zipfile
import gspread
import matplotlib.pyplot as plt
from oauth2client.service_account import ServiceAccountCredentials
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score
from skimage import color, exposure, feature, io, transform

import numpy as np
import cv2
# from matplotlib import pyplot as plt

# setup_run = True
# data_base_path = 'https://ait.ethz.ch/public-data/computational_interaction2016/'

# if not os.path.exists('train'):
#     print('[INFO]: Looks like you do not have training data. Let me fetch that for you.')
#     sys.stdout.flush()
#     url_traindata = data_base_path+'train.zip'
#     filename = wget.download(url_traindata)
#     zip_ref = zipfile.ZipFile(filename, 'r')
#     zip_ref.extractall('./')
#     zip_ref.close()
#     print('[INFO]: Training data fetching completed.')
#     sys.stdout.flush()
    
# if not os.path.exists('./test_T30_R60'):
#     print('[INFO]: Looks like you do not have testing data. Let me fetch that for you')
#     sys.stdout.flush()
#     url_testdata = data_base_path+'test_T30_R60.zip'
#     filename = wget.download(url_testdata)
#     zip_ref = zipfile.ZipFile(filename, 'r')
#     zip_ref.extractall('./')
#     zip_ref.close()
#     print('[INFO]: Testing data fetching completed.')
#     sys.stdout.flush()
    
# Additionally, there's a second, more challenging dataset that you can download from 
# url_testdata_hard = 'https://ait.inf.ethz.ch/teaching/courses/2016-SS-User-Interface-Engineering/downloads/exercises/test_T30_R90.zip '
    
# Compute accuracy, precision, recall and confusion matrix and (optionally) prints them on screen
def compute_scores(y_pred, y_true, verbose=False):

    hits = 0
    for p in range(1,len(y_true)):
        if y_pred[p] == y_true[p]:
            hits += 1

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='macro')
    recall = recall_score(y_true, y_pred, average='macro')
    conf_mat = confusion_matrix(y_true, y_pred)

    if(verbose):
        print ("(RW) Accuracy: " + str(accuracy) + "(" + str(hits) + "/" + str(len(y_true)) + ")")
        print ("Precision: " + str(precision))
        print ("Recall: " + str(recall))
        print ("Confusion Matrix")
        print (conf_mat)
        sys.stdout.flush()

    return accuracy, precision, recall


# Extract HOG features from an image and (optionally) show the features superimposed on it 
def extractHOG(inputimg, showHOG=False): 
    
    # convert image to single-channel, grayscale
    image = color.rgb2gray(inputimg)

    #extract HOG features
    if showHOG:
        fd, hog_image = feature.hog(image, orientations=36, 
                                    pixels_per_cell=(16, 16),
                                    cells_per_block=(2, 2), 
                                    visualise=showHOG)
    else:
        fd = feature.hog(image, orientations=8, pixels_per_cell=(16, 16),
                         cells_per_block=(1, 1), visualise=showHOG)
    if(showHOG):
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4), sharex=True, sharey=True)
        ax1.axis('off')
        ax1.imshow(image, cmap=plt.cm.gray)
        ax1.set_title('Input image')
        ax1.set_adjustable('box-forced')
        # Rescale histogram for better display
        hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 0.02))
        ax2.axis('off')
        ax2.imshow(hog_image_rescaled, cmap=plt.cm.gray)
        ax2.set_title('Histogram of Oriented Gradients')
        ax1.set_adjustable('box-forced')
        plt.show()
    return fd


# Load a dataset (Data, Labels) from a folder.
# Return data (HOGs, Class) and image list (as image file ames on disk)
def load_dataset_from_folder(root_folder, rgb_folder, segmentation_folder):
            
    HOGs_list = []
    Cs_list = []    
    image_list = []
    if os.path.exists(root_folder):
        class_folders = next(os.walk(root_folder))[1]
        class_folders.sort()
        print("[INFO] Found " + str(len(class_folders)) + " class folders")
        print(class_folders)
        sys.stdout.flush()
        tot_classes = len(class_folders)
        #used to resize the images
        image_size = (128, 128)
        class_list = range(tot_classes)
        for class_folder,this_class in zip(class_folders,class_list):
            print("\n[INFO] Processing folder " + class_folder)
            sys.stdout.flush()
            current_gesture_folder_rgb = root_folder + class_folder + "/" + rgb_folder + "/*.jpg"
            current_gesture_folder_segmentation = root_folder + class_folder + "/" + segmentation_folder + "/*.png"
            allfiles_imgs = glob.glob(current_gesture_folder_rgb)
            allfiles_masks = glob.glob(current_gesture_folder_segmentation)
            #for each image/mask pair
            line_percentage_cnt = 0
            for file_img,mask_img in zip(allfiles_imgs,allfiles_masks):
                # Print completion percentage
                sys.stdout.write('\r')
                progress_bar_msg = "[%-100s] %d%% " + str(line_percentage_cnt) + "/" + str(len(allfiles_imgs))
                update_step = int( (float(100)/float(len(allfiles_imgs))) * float(line_percentage_cnt) )
                sys.stdout.write(progress_bar_msg % ('='*update_step, update_step))
                sys.stdout.flush()
                img = io.imread(file_img)
                mask = io.imread(mask_img)
                mask = 255 - mask
                img *= mask
                # you can see the segmented image using:
                #io.imshow(img)
                #io.show()
                
                feat = extractHOG(transform.resize(img, image_size))
                HOGs_list.append(feat)
                Cs_list.append(this_class)
                image_list.append(file_img)
                line_percentage_cnt += 1
        print("[INFO] Loaded data in. Number of samples: "+ str(len(image_list)))
    else:
        print("[ERROR] Folder " + root_folder + " does not exist!")
        print("[ERROR] Have you run the setup cell?")
        sys.stdout.flush()
        exit()
        

    HOGs = np.array(HOGs_list)
    Cs = np.array(Cs_list)
    return HOGs, Cs, image_list


        
# Class to store parameters of an SVM
class SVMparameters:

    def __init__(self, k='rbf', c='1', g='0.1', d=1):
        self.kernel = k
        self.C = c
        self.gamma=g
        self.degree = g

    def setkernel(self, k):
        self.kernel = k

    def setgamma(self, g):
        self.gamma = g

    def setc(self, c):
        self.C = c

    def setdegree(self,d):
        self.degree = d
    
    def printconfig(self):
        print("Kernel: " + self.kernel)
        if self.kernel is "poly":
            print("Degree: " + str(self.degree))
        print("C: " + str(self.C))
        print("Gamma: " + str(self.gamma))
        sys.stdout.flush()

In [28]:
train_folders = next(os.walk("KDEF/"))[1]
train_folders.sort()
print("[INFO] Found " + str(len(train_folders)) + " class folders")
sys.stdout.flush()


image_list = []
HOGs_list = []
Cs_list = [] 

image_size = (128, 128)
                

face_cascade = cv2.CascadeClassifier('FaceDetect/haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('FaceDetect/haarcascade_eye.xml')

emotion_list = ['AFS.JPG', 'ANS.JPG', 'DIS.JPG', 'HAS.JPG', 'NES.JPG', 'SAS.JPG', 'SUS.JPG']

emotion_num = [0,1,2,3,4,5,6]

ffile = []
line_percentage_cnt = 0

for class_folder,this_class in zip(train_folders,class_list):
#     print("\n[INFO] Processing folder " + class_folder)
    sys.stdout.flush()
  
    
    ffile = os.listdir('KDEF/' + class_folder)

    for i in ffile:
        
        for j in range(len(emotion_list)):

            if i.endswith(emotion_list[j]): #afraid
                
                Cs_list.append(emotion_num[j])
                
                print Cs_list
                
                directory = 'KDEF/train/' + emotion_list[j][0:3] 
                
                RGB_subdir = 'RGB/'
                segmented_subdir = 'segmented/'
                
                if not os.path.exists(directory):
                    os.makedirs(directory)
                     
                if not os.path.exists(directory + "/" + RGB_subdir):
                    os.makedirs(directory + "/" + RGB_subdir)
                
              
                tmp_img = io.imread("KDEF/" + class_folder + "/" + i)

                
            
#                 sys.stdout.write('\r')
#                 progress_bar_msg = "[%-100s] %d%% " + str(line_percentage_cnt) + "/" + str(len(class_folder))
#                 update_step = int( (float(100)/float(len(allfiles_imgs))) * float(line_percentage_cnt) )
#                 sys.stdout.write(progress_bar_msg % ('='*update_step, update_step))
#                 sys.stdout.flush()

#                 feat = extractHOG(img_resize)
        
#                 HOGs_list.append(feat)
    #             Cs_list.append(this_class)
#                 image_list.append(tmp_img)

#                 img = cv2.imread("KDEF/AF01/AF01AFFL.JPG")
                gray = cv2.cvtColor(tmp_img,cv2.COLOR_BGR2GRAY)
                ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
        
        
                ###########################
   
                faces = face_cascade.detectMultiScale(gray, 1.3, 5)
                for (x,y,w,h) in faces:
                    cv2.rectangle(tmp_img,(x,y),(x+w,y+h),(255,0,0),2)
                    roi_gray = gray[y:y+h, x:x+w]
                    roi_color = tmp_img[y:y+h, x:x+w]
                    eyes = eye_cascade.detectMultiScale(roi_gray)
#                     mouth = mouth_cascade.detectMultiScale(roi_gray)
                    for (ex,ey,ew,eh) in eyes:
                        cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)

                img_resize = tmp_img[y:y+h,x:x+w]

                
                io.imsave(directory + "/" + RGB_subdir + i, img_resize)
                
                ##########################

# #                 # noise removal
#                 kernel = np.ones((3,3),np.uint8)
#                 opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)

# #                 # sure background area
#                 sure_bg = cv2.dilate(opening,kernel,iterations=3)

# #                 # Finding sure foreground area
#                 dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
#                 ret, sure_fg = cv2.threshold(dist_transform,0.7*dist_transform.max(),255,0)

# #                 # Finding unknown region
#                 sure_fg = np.uint8(sure_fg)
#                 mask_img = cv2.subtract(sure_bg,sure_fg)

#                 if not os.path.exists(directory + "/" + segmented_subdir):
#                     os.makedirs(directory + "/" + segmented_subdir)
                    
# #               #  cv2.imwrite('KDEF/test.png', mask_img)
#                 io.imsave(directory + "/" + segmented_subdir + i, mask_img)

#                 mask = 255 - mask_img
#                 gray *= mask 
                feat = extractHOG(transform.resize(gray, image_size))

                HOGs_list.append(feat)
#                 Cs_list.append(this_class)
                image_list.append(img_resize)
                

        
HOGs = np.array(HOGs_list)
Cs = np.array(Cs_list)


    
#     return HOGs, Cs, image_list


[INFO] Found 141 class folders


In [27]:

print Cs.shape
print feat.shape

(980,)
(13160,)


In [4]:

face_cascade = cv2.CascadeClassifier('FaceDetect/haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('FaceDetect/haarcascade_eye.xml')
# mouth_cascade = cv2.CascadeClassifier('FaceDetect/haarcascade_smile.xml')


img = cv2.imread('KDEF/AF01/AF01AFS.JPG')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)


faces = face_cascade.detectMultiScale(gray, 1.3, 5)
for (x,y,w,h) in faces:
    cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]
    eyes = eye_cascade.detectMultiScale(roi_gray)
#     mouth = mouth_cascade.detectMultiScale(roi_gray)
    for (ex,ey,ew,eh) in eyes:
        cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
    
#     for (ex,ey,ew,eh) in mouth:
#         cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)

height, width = img.shape[:2]

        
img_resize = img[y:y+h,x:x+w]


print img_resize.shape

# io.imshow(img_resize)
# io.show()

# cv2.imshow('img',img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


(379, 379, 3)


In [ ]:
import cv2
from cv2 import xfeatures2d

xfeatures2d


In [30]:

image_size = (128, 128)
# img = cv2.imread("KDEF/AF01/AF01AFFL.JPG")

img = cv2.imread("KDEF/AF01/AF01AFFL.JPG")
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

# noise removal
kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)

# sure background area
sure_bg = cv2.dilate(opening,kernel,iterations=3)

# Finding sure foreground area
dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
ret, sure_fg = cv2.threshold(dist_transform,0.7*dist_transform.max(),255,0)

# Finding unknown region
sure_fg = np.uint8(sure_fg)
mask_img = cv2.subtract(sure_bg,sure_fg)

cv2.imwrite('KDEF/test.png', mask_img)

# io.imshow(mask_img)
# io.show()

# img = io.imread("KDEF/AF01/AF01AFFL.JPG")
mask = io.imread("KDEF/test.png")
mask = 255 - mask

# print mask.shape
# print gray.shape
# (762, 562, 3)
# 388 rows, 647 columns, and 3 channels (the RGB components).

gray *= mask
feat = extractHOG(transform.resize(img, image_size))
extractHOG(img, showHOG=False)


(762, 562)
(762, 562)


array([ 0.14695973,  0.1065591 ,  0.06730217, ...,  0.08589244,
        0.11436558,  0.10563707])

In [ ]:
img = io.imread("KDEF/AF01/AF01AFFL.JPG")
mask = io.imread(mask_img
mask = 255 - mask
img *= mask
# you can see the segmented image using:
#io.imshow(img)
#io.show()
feat = extractHOG(transform.resize(img, image_size))

extractHOG(img, showHOG=True)

In [ ]:

from window import window_data

## The skeleton of a solution

import window
import numpy as np
import sklearn.datasets, sklearn.linear_model, sklearn.neighbors
import matplotlib.pyplot as plt
#import seaborn as sns
import sys, os, time
import scipy.io.wavfile, scipy.signal
#%matplotlib inline
import matplotlib as mpl
from IPython.core.display import HTML
mpl.rcParams['figure.figsize'] = (18.0, 10.0)
import pandas as pd




# Process

# - get external labeled facial images from available datasets
# - image database: extract relevant features, e.g., color histograms, contours, etc. (use off shelf packages)
# - features split into training and test data
# - classifier must be trained on extracted training features

# - Voice input: “emoji” command, starts window function (e.g., 200 ms), draws randomly sampled snapshot
# - snapshot used as input to trained classifier (e.g., scikit.svm.predict(snapshot)). returns label (happy, sad, neutral, angry)
# - get label and input image, and convert image to cartoon. store final cartoon image into database, with associated label. this gives personalized emoji database
# - user says “emoji”, and if facial features matches existing emoji in database
# - two windows, one with your face and other with output emoji

# corner features, sift features, gabor features






# load the wave file and normalise
# load "data/rub_1.wav" and "data/rub_2.wav"

def load_wave(fname):
    # load and return a wave file
    sr, wave = scipy.io.wavfile.read(fname)
    return wave/32768.0



def breath_classify():

    rub_1 = load_wave("data/in.wav")[:,0]
    rub_2 = load_wave("data/out.wav")[:,0]

    rub_1_features = window.window_data(rub_1, 120)
    rub_2_features = window.window_data(rub_2, 120)

    rub_1_labels = np.zeros(len(rub_1_features,))
    rub_2_labels = np.ones(len(rub_2_features,))

    rub_features = np.vstack([rub_1_features, rub_2_features])
    rub_labels = np.hstack([rub_1_labels, rub_2_labels])
    print rub_features.shape, rub_labels.shape

    rubfft_features =  np.abs(np.fft.fft(rub_features))
    rubfft_train_features, rubfft_test_features, rub_train_labels, rub_test_labels = sklearn.cross_validation.train_test_split(
        rubfft_features, rub_labels, test_size=0.3, random_state=0)

    rub_train_features, rub_test_features, rub_train_labels, rub_test_labels = sklearn.cross_validation.train_test_split(
        rub_features, rub_labels, test_size=0.3, random_state=0)

    print rub_train_features.shape, rub_train_labels.shape

    svm = sklearn.svm.SVC(gamma=0.1, C=100)
    svm.fit(rub_train_features, rub_train_labels)

    # we can plot the receiver-operator curve: the graph of false positive rate against true positive rate
    # scores = svm.decision_function(rub_test_features)
    # print scores.shape, rub_test_labels.shape
    # fpr, tpr, thresholds = sklearn.metrics.roc_curve(rub_test_labels, scores)
    # plt.plot(fpr,tpr)
    # plt.plot([0,1], [0,1])
    # plt.plot([0,1], [1,0])
    # plt.fill_between(fpr, tpr, facecolor='none', hatch='/', alpha=0.2)
    # plt.xlabel("False positive rate")
    # plt.ylabel("True positive rate")
    # plt.legend(["ROC", "Chance", "EER line"])

    # print svm

    return svm


breath_classify()
 # takes feature vector and returns a label


# split into windows

# test/train split

# train classifier

# evaluate classifier

